## (0) Test Python functionality

In [1]:
print('hey', 9+6)
import os
print(os.uname())

hey 15
posix.uname_result(sysname='Linux', nodename='cn3095', release='3.10.0-862.14.4.el7.x86_64', version='#1 SMP Wed Sep 26 15:12:11 UTC 2018', machine='x86_64')


# Main workflow

## (1) Set the project directory and load the main Python library

In [1]:
project_dir = '/data/BIDS-HPC/private/projects/dmi'
import os, sys
checkout_dir = os.path.join(project_dir, 'checkout')
if checkout_dir not in sys.path:
    sys.path.append(checkout_dir)
import target_class_lib as tc

## (2) Generate or read in the initial lookup lists that can be calculated by querying the Ensembl REST API

In [2]:
symbol_lookup_list, id_lookup_list = tc.get_initial_lookup_lists(project_dir)

Reading pickle file /data/BIDS-HPC/private/projects/dmi/data/initial_lookup_lists.pkl...


## (3) Go through the missing lookups in the symbols lookup list and try to determine them using the xref endpoint of the Ensembl REST API

In [5]:
missing_lookup_list = tc.get_missing_lookups(project_dir, symbol_lookup_list, max_num_names=5)

Reading pickle file /data/BIDS-HPC/private/projects/dmi/data/missing_lookup_list.pkl...


# Scratch

In [6]:
missing_lookup_list

[['AC005391.3', None],
 ['AC005330.2', None],
 ['AC005329.7', None],
 ['AC007040.10', None],
 ['AC006159.4', None]]

In [ ]:
lookup_list = []
iiter = 0
nidentified = 0
for item in symbol_lookup_list:
    if item[1] is None:
        lookup_list, result_processed, iiter, nidentified = run_and_process_query('xref', [item[0]], lookup_list, iiter, nidentified)

In [49]:
#res = tc.ensembl_name_request('xref', [to_xref2[0]])
#res = tc.ensembl_name_request('xref', ['AC005086.1'])
#res
names_list = ['AC005086.1']
res = [{'type': 'gene', 'id': 'ENSG00000252824'}]
{names_list[0]: res[0]}

if endpoint == 'xref':
    if res:
        #print('not empty')
        res = {names_list[0]: res[0]}
    else:
        #print('empty')
        res = {}

# Symbol does not exist: {}
# Symbol exists:
# {'AC005086.1': {'version': 1,
#   'biotype': 'snoRNA',
#   'object_type': 'Gene',
#   'strand': -1,
#   'id': 'ENSG00000252824',
#   'db_type': 'core',
#   'assembly_name': 'GRCh38',
#   'logic_name': 'ncrna_homo_sapiens',
#   'source': 'ensembl',
#   'display_name': 'AC005086.1',
#   'seq_region_name': '7',
#   'end': 102194164,
#   'species': 'homo_sapiens',
#   'start': 102194076}}

# xref does not exist: [] --> {}
# xref exists: [{'type': 'gene', 'id': 'ENSG00000252824'}] --> {names_list[0]: res[0]}

ERROR: Names list is too long (2) for the xref endpoint (max length is 1)


NameError: name 'exit' is not defined

In [48]:
'andrew' == 'anddrew'

False

In [50]:
tmp_list = []
iiter = 0
nidentified = 0
for item in to_xref2:
    tmp_list, result_processed, iiter, nidentified = tc.run_and_process_query('xref', [item], tmp_list, iiter, nidentified)

AttributeError: 'list' object has no attribute 'keys'

In [13]:
to_xref2

['AC005391.3',
 'AC005330.2',
 'AC005329.7',
 'AC007040.10',
 'AC006159.4',
 'AC004893.11',
 'AC004692.4',
 'AC002310.11',
 'AC005392.13',
 'AC002069.6']

In [12]:
to_xref2 = [ to_xref[x] for x in range(10) ]

In [5]:
symbol_lookup_list

'ENSG00000198691'],
 ['AC005280.1', 'ENSG00000251393'],
 ['AC005086.1', 'ENSG00000252824'],
 ['ABHD11-AS1', 'ENSG00000225969'],
 ['A1CF', 'ENSG00000148584'],
 ['AADACL2', 'ENSG00000197953'],
 ['AC006427.1', 'ENSG00000248851'],
 ['AC007179.1', 'ENSG00000231815'],
 ['AASS', 'ENSG00000008311'],
 ['ABCC5-AS1', 'ENSG00000223882'],
 ['A2MP1', 'ENSG00000256069'],
 ['AATF', 'ENSG00000275700'],
 ['AC006427.2', 'ENSG00000286888'],
 ['ABCG4', 'ENSG00000172350'],
 ['AC005013.1', 'ENSG00000228421'],
 ['AC005086.3', 'ENSG00000279724'],
 ['ABCC11', 'ENSG00000121270'],
 ['AC002331.1', 'ENSG00000286712'],
 ['AC003070.1', 'ENSG00000267344'],
 ['ABHD17A', 'ENSG00000129968'],
 ['AC002553.1', 'ENSG00000227782'],
 ['ABHD11', 'ENSG00000106077'],
 ['AC005544.1', 'ENSG00000264491'],
 ['ABCC6P1', 'ENSG00000256340'],
 ['ABCA9', 'ENSG00000154258'],
 ['ABCC2', 'ENSG00000023839'],
 ['AC006076.1', 'ENSG00000225563'],
 ['AC006539.1', 'ENSG00000240673'],
 ['ABHD6', 'ENSG00000163686'],
 ['ABHD8', 'ENSG00000127220'],
 [

In [10]:
to_xref = []
for item in symbol_lookup_list:
    if item[1] is None:
        to_xref.append(item[0])

In [3]:
if not os.path.exists(os.path.join(project_dir,'data','primary_lookup_lists.pkl')):

    # Import the GMB library in order to get the make_pickle() function
    gmb_dir = '/data/BIDS-HPC/private/projects/gmb/checkout'
    if gmb_dir not in sys.path:
        sys.path.append(gmb_dir)
    import time_cell_interaction_lib as tci # we need this to get the make_pickle() function

    # Query the Ensembl REST API to see whether we can identify the gene "names"
    id_lookup_list, id_nidentified = tc.get_lookup_list('id', os.path.join(project_dir,'data','unique_ensembl_ids.txt'))
    symbol_lookup_list, symbol_nidentified = tc.get_lookup_list('symbol', os.path.join(project_dir,'data','unique_other_names.txt'))

    # Run a check on the number of Nones in the lists; otherwise, save the lookup lists to disk
    if (sum([ 0 if x[1] is None else 1 for x in symbol_lookup_list ]) != symbol_nidentified) or (sum([ 0 if x[1] is None else 1 for x in id_lookup_list ]) != id_nidentified):
        print('ERROR: Inconsistent number of Nones in the lookup lists')
        exit
    else:
        tci.make_pickle([symbol_lookup_list, id_lookup_list], os.path.join(project_dir,'data'), 'primary_lookup_lists.pkl')

else:
    [symbol_lookup_list, id_lookup_list] = tci.load_pickle(os.path.join(project_dir,'data'), 'primary_lookup_lists.pkl')

[1] Given 1000 names, we identified 987 of them
[2] Given 1000 names, we identified 989 of them
[3] Given 1000 names, we identified 990 of them
[4] Given 1000 names, we identified 984 of them


KeyboardInterrupt: 

In [4]:
id_lookup_list, id_nidentified = tc.get_lookup_list('id', os.path.join(project_dir,'data','unique_ensembl_ids.txt'), max_list_len=10, max_iter=15)

[1] Given 10 names, we identified 10 of them
[2] Given 10 names, we identified 10 of them
[3] Given 10 names, we identified 10 of them
[4] Given 10 names, we identified 10 of them
[5] Given 10 names, we identified 10 of them
[6] Given 10 names, we identified 10 of them
[7] Given 10 names, we identified 10 of them
[8] Given 10 names, we identified 10 of them
[9] Given 10 names, we identified 10 of them
[10] Given 10 names, we identified 10 of them
[11] Given 10 names, we identified 10 of them
[12] Given 10 names, we identified 9 of them
[13] Given 10 names, we identified 7 of them
[14] Given 10 names, we identified 10 of them
[15] Given 10 names, we identified 10 of them


In [22]:
sum([ 0 if x[1] is None else 1 for x in symbol_lookup_list ])

870

In [17]:
col.is

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [7]:
list1 = ["ENSG00000005483", "ENSG00000005486", "ENSG00000005513", "ENSG00000005700", "ENSG00000005801", "ENSG00000005810", "ENSG00000005812", "ENSG00000005844", "ENSG00000005882", "ENSG00000005884"]
list2 = ["ENSG00000005889", "ENSG00000005893", "ENSG00000005955", "ENSG00000005961", "ENSG00000005981", "ENSG00000006007", "ENSG00000006015", "ENSG00000006016", "ENSG00000006025", "ENSG00000006042"]
res1 = tc.ensembl_name_request('id', list1)
res2 = tc.ensembl_name_request('id', list2)

In [9]:
res2

{'ENSG00000005961': {'db_type': 'core',
  'assembly_name': 'GRCh38',
  'id': 'ENSG00000005961',
  'start': 44372180,
  'logic_name': 'ensembl_havana_gene_homo_sapiens',
  'source': 'ensembl_havana',
  'end': 44389649,
  'display_name': 'ITGA2B',
  'seq_region_name': '17',
  'species': 'homo_sapiens',
  'biotype': 'protein_coding',
  'version': 19,
  'strand': -1,
  'description': 'integrin subunit alpha 2b [Source:HGNC Symbol;Acc:HGNC:6138]',
  'object_type': 'Gene'},
 'ENSG00000006025': {'description': 'oxysterol binding protein like 7 [Source:HGNC Symbol;Acc:HGNC:16387]',
  'strand': -1,
  'object_type': 'Gene',
  'biotype': 'protein_coding',
  'version': 12,
  'start': 47807372,
  'source': 'ensembl_havana',
  'display_name': 'OSBPL7',
  'end': 47821834,
  'species': 'homo_sapiens',
  'seq_region_name': '17',
  'logic_name': 'ensembl_havana_gene_homo_sapiens',
  'assembly_name': 'GRCh38',
  'db_type': 'core',
  'id': 'ENSG00000006025'},
 'ENSG00000006042': {'id': 'ENSG00000006042',


In [5]:
symbol_lookup_list

ENSG00000287636'],
 ['AARD', 'ENSG00000205002'],
 ['AC002066.1', 'ENSG00000237813'],
 ['AC004148.1', 'ENSG00000263272'],
 ['ABCB6', 'ENSG00000115657'],
 ['AC006947.1', 'ENSG00000229330'],
 ['AC003984.1', 'ENSG00000235139'],
 ['AC006499.2', 'ENSG00000250074'],
 ['AC003006.1', 'ENSG00000269026'],
 ['AC005537.2', 'ENSG00000232006'],
 ['AANAT', 'ENSG00000129673'],
 ['ABL2', 'ENSG00000143322'],
 ['AC005625.1', 'ENSG00000266963'],
 ['AC004771.1', 'ENSG00000234203'],
 ['ABCC9', 'ENSG00000069431'],
 ['ABI2', 'ENSG00000138443'],
 ['AC000124.1', 'ENSG00000226770'],
 ['ABHD13', 'ENSG00000139826'],
 ['AC005943.2', 'ENSG00000279009'],
 ['ABI3BP', 'ENSG00000154175'],
 ['AC002511.1', 'ENSG00000232680'],
 ['AC004063.1', 'ENSG00000250670'],
 ['AC003043.2', 'ENSG00000267211'],
 ['AC004562.1', 'ENSG00000199801'],
 ['AC005307.1', 'ENSG00000260725'],
 ['AC005154.5', 'ENSG00000281039'],
 ['ABCD1', 'ENSG00000101986'],
 ['AC002480.2', 'ENSG00000232949'],
 ['AC006328.1', 'ENSG00000233619'],
 ['AC004079.1', 'EN

## (2) Return a list of symbols and corresponding IDs identified using the symbol lookup REST function of Ensembl given the symbols in a text file called to_lookup.txt

In [2]:
max_post_size = 1000
res = tc.get_lookup_list(project_dir, max_post_size=max_post_size)

[1] Given 1000 symbols, we identified 311 of them
[2] Given 1000 symbols, we identified 477 of them
[3] Given 1000 symbols, we identified 457 of them
[4] Given 1000 symbols, we identified 462 of them
[5] Given 1000 symbols, we identified 470 of them
[6] Given 1000 symbols, we identified 424 of them
[7] Given 1000 symbols, we identified 180 of them
[8] Given 1000 symbols, we identified 82 of them
[9] Given 1000 symbols, we identified 0 of them
[10] Given 1000 symbols, we identified 22 of them
[11] Given 1000 symbols, we identified 17 of them
[12] Given 1000 symbols, we identified 0 of them
[13] Given 1000 symbols, we identified 10 of them
[14] Given 1000 symbols, we identified 0 of them
[15] Given 1000 symbols, we identified 0 of them
[16] Given 1000 symbols, we identified 0 of them
[17] Given 1000 symbols, we identified 0 of them
[18] Given 1000 symbols, we identified 0 of them
[19] Given 1000 symbols, we identified 0 of them
[20] Given 1000 symbols, we identified 0 of them
[21] Given 

## (3) Load the lookup table, add the new entries to it, and save it to disk

In [34]:
# Import relevant modules
import pandas as pd
import numpy as np
gmb_dir = '/data/BIDS-HPC/private/projects/gmb/checkout'
if gmb_dir not in sys.path:
    sys.path.append(gmb_dir)
import time_cell_interaction_lib as tci # we need this to get the make_pickle() function

# Load the first-pass of the global lookup table using Pandas
lookup = pd.read_csv(os.path.join(project_dir,'data','partial_global_lookup_table.txt'), sep='\t', names=['status','name','id'], index_col='name')

# Count how many null IDs there are
nnull_before = np.sum(pd.isnull(lookup.loc[:,'id']))

# Use the list generated in Step (2) to add IDs to the lookup table
for pair in res:
    lookup.loc[pair[0],:] = ['lookup:', pair[1]]

# Count how many null IDs there are
nnull_after = np.sum(pd.isnull(lookup.loc[:,'id']))

# Print how we improved the lookup table
print('{} lookup pairs have been recorded; {} null before, {} null after'.format(len(res), nnull_before, nnull_after))

# Save the new lookup table in the data directory
tci.make_pickle(lookup, os.path.join(project_dir,'data'), 'lookup_table_after_symbol_lookup.pkl')

2948 lookup pairs have been recorded; 27695 null before, 24747 null after
Creating pickle file /data/BIDS-HPC/private/projects/dmi/data/lookup_table_after_symbol_lookup.pkl...


In [ ]:
lookup = tci.load_pickle(os.path.join(project_dir,'data'), 'lookup_table_after_symbol_lookup.pkl')

In [7]:
set(['a','c','d','e','f']) - set(['a','b','d'])


{'c', 'e', 'f'}

In [35]:
bleh = tci.load_pickle(os.path.join(project_dir,'data'), 'lookup_table_after_symbol_lookup.pkl')

Reading pickle file /data/BIDS-HPC/private/projects/dmi/data/lookup_table_after_symbol_lookup.pkl...


In [41]:
lookup.equals(bleh)

True

In [31]:
np.sum(lookup.loc[:,'status'] == 'unmatched:')

24747

In [13]:
lookup

,status,id
name,,
ENSG00000000003,lookup:,ENSG00000000003
ENSG00000000005,lookup:,ENSG00000000005
ENSG00000000419,lookup:,ENSG00000000419
ENSG00000000457,lookup:,ENSG00000000457
ENSG00000000460,lookup:,ENSG00000000460
...,...,...
ZNRD1,unmatched:,NaN
ZNRD1-AS1,unmatched:,NaN
ZRSR1,unmatched:,NaN


In [27]:
print(np.sum(pd.isnull(lookup.loc[:,'id'])))

27695


In [32]:
#lookup.loc['ENSG00000000457','id'] = 'andrew'
lookup.loc['AC010376.1',:]

status            lookup:
id        ENSG00000253801
Name: AC010376.1, dtype: object

In [11]:
import numpy as np
#np.sum(lookup['id'] == '')
#np.sum(pd.isnull(lookup.loc[:,'id']))
lookup.loc['AC009161.1']

KeyError: 'AC009161.1'

In [3]:
print(len(res))
res

2948


05'],
 ['AC009474.2', 'ENSG00000230355'],
 ['AC013410.2', 'ENSG00000237617'],
 ['AC009161.1', 'ENSG00000237901'],
 ['AC011379.1', 'ENSG00000250069'],
 ['AC016727.1', 'ENSG00000270820'],
 ['AC012564.1', 'ENSG00000283157'],
 ['AC012314.7', 'ENSG00000274905'],
 ['AC009492.1', 'ENSG00000278004'],
 ['AC011551.1', 'ENSG00000278797'],
 ['AC008565.1', 'ENSG00000251135'],
 ['AC009065.2', 'ENSG00000259780'],
 ['AC011453.1', 'ENSG00000269842'],
 ['AC013268.5', 'ENSG00000283684'],
 ['AC012462.3', 'ENSG00000230695'],
 ['AC009800.1', 'ENSG00000248911'],
 ['AC007948.1', 'ENSG00000263863'],
 ['AC010970.1', 'ENSG00000225840'],
 ['AC010376.1', 'ENSG00000253801'],
 ['AC008132.1', 'ENSG00000161103'],
 ['AC015871.1', 'ENSG00000259770'],
 ['AC008746.5', 'ENSG00000276551'],
 ['AC008410.1', 'ENSG00000253261'],
 ['AC008734.1', 'ENSG00000269300'],
 ['AC009478.1', 'ENSG00000225258'],
 ['AC015987.1', 'ENSG00000224746'],
 ['AC008674.1', 'ENSG00000287003'],
 ['AC016734.1', 'ENSG00000228305'],
 ['AC011752.1', 'ENSG0

In [30]:
datadir = os.path.join(project_dir, 'data')
max_post_size = 20
max_iter = 5
iter = 0
lookup_list = []
with open(os.path.join(datadir, 'to_lookup.txt'), 'r') as f:
    iname = 0
    symbols_list = []
    for name in f:
        iname = iname + 1
        symbols_list.append(name.rstrip())
        if iname == max_post_size:
            res = tc.ensembl_symbol_lookup(symbols_list)
            for key in res.keys():
                lookup_list.append([key, res.get(key).get('id')])
            iter = iter + 1
            if iter == max_iter:
                break
            iname = 0
            symbols_list = []

if iname != max_post_size:
    res = tc.ensembl_symbol_lookup(symbols_list)
    for key in res.keys():
        lookup_list.append([key, res.get(key).get('id')])
    iter = iter + 1


print(iname)
print(symbols_list)
#tc.ensembl_symbol_lookup(symbols_list)

20
['AC002310.11', 'AC002310.12', 'AC002310.13', 'AC002310.17', 'AC002310.7', 'AC002314.2', 'AC002314.4', 'AC002321.1', 'AC002321.2', 'AC002331.1', 'AC002365.1', 'AC002366.3', 'AC002368.4', 'AC002383.2', 'AC002386.1', 'AC002395.1', 'AC002398.11', 'AC002398.12', 'AC002398.13', 'AC002398.5']


In [32]:
len(lookup_list)

17

In [6]:
res = tc.ensembl_symbol_lookup(symbols_list)
# # The maximum POST size (and therefore list size) is 1000
# if len(symbols_list) > 1000:
#     print('ERROR: POST lookup list is too long ({})'.format(len(symbols_list)))
#     exit

# # Import relevant modules
# import requests, sys

# # Set some constants
# server = "https://rest.ensembl.org"
# ext = "/lookup/symbol/homo_sapiens"
# headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

# # Query the REST server
# #r = requests.post(server+ext, headers=headers, data='{ "symbols" : ["BRCA2", "BRAF" ] }')
# r = requests.post(server+ext, headers=headers, data=str({'symbols': symbols_list}).replace('\'','"'))

# # Ensure everything is okay
# if not r.ok:
#     r.raise_for_status()
#     sys.exit()

# # Return the result
# #return(repr(r.json()))
# print(r.json())


In [27]:
print([ [key, res.get(key).get('id')] for key in res.keys() ])
#res.get('AC000124.1')

[['AC002386.1', 'ENSG00000230941'], ['AC000362.1', 'ENSG00000230820'], ['AC002056.3', 'ENSG00000283252'], ['AC002306.1', 'ENSG00000259242'], ['AC000124.1', 'ENSG00000226770'], ['AC002066.1', 'ENSG00000237813'], ['AC000081.2', 'ENSG00000251940'], ['AC002070.1', 'ENSG00000248636'], ['AC000367.1', 'ENSG00000224981'], ['AC000374.1', 'ENSG00000227249'], ['AC002064.4', 'ENSG00000280440'], ['AC002395.1', 'ENSG00000272215'], ['AC000029.1', 'ENSG00000275635'], ['AC000067.1', 'ENSG00000225007'], ['AC002365.1', 'ENSG00000206844'], ['AC002331.1', 'ENSG00000286712'], ['AC000403.1', 'ENSG00000278727']]


In [39]:
# The maximum POST size (and therefore list size) is 1000
if len(symbols_list) > 1000:
    print('ERROR: POST lookup list is too long ({})'.format(len(symbols_list)))
    exit

# Import relevant modules
import requests, sys

# Set some constants
server = "https://rest.ensembl.org"
ext = "/lookup/symbol/homo_sapiens"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

# Query the REST server
#r = requests.post(server+ext, headers=headers, data='{ "symbols" : ["BRCA2", "BRAF" ] }')
r = requests.post(server+ext, headers=headers, data=str({'symbols': symbols_list}).replace('\'','"'))

# Ensure everything is okay
if not r.ok:
    r.raise_for_status()
    sys.exit()

# Return the result
print(r.json())


{}


In [38]:
#str({'symbols': symbols_list}).replace('\'','"')
#
mystr = '{"symbols": ["1060P11.3", "7SK", "A2LD1", "A3GALT2P", "AA06", "AACP", "AAED1", "AARS", "AATK-AS1", "AB015752.3"]}'
r = requests.post(server+ext, headers=headers, data=mystr)
#[ repr(x) for x in r.iter_lines() ]
r.json()

{}

In [4]:
# The maximum POST size (and therefore list size) is 1000
if len(symbols_list) > 1000:
    print('ERROR: POST lookup list is too long ({})'.format(len(symbols_list)))
    exit

# Import relevant modules
import requests, sys

# Set some constants
server = "https://rest.ensembl.org"
ext = "/lookup/symbol/homo_sapiens"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

# # Query the REST server
# #r = requests.post(server+ext, headers=headers, data='{ "symbols" : ["BRCA2", "BRAF" ] }')
r = requests.post(server+ext, headers=headers, data=str({'symbols': symbols_list}))


# # Ensure everything is okay
# if not r.ok:
#     r.raise_for_status()
#     sys.exit()

# # Return the result
# return(repr(r.json()))


In [17]:
tmp = { 'symbols' : ['BRCA2', 'BRAF' ] }
mydata = str(tmp).replace('\'','"')
r = requests.post(server+ext, data=mydata)

In [18]:
[ print(x) for x in r.iter_lines() ]

b'<html><title>EnsEMBL::REST</title><body><pre>--- '
b'BRAF: '
b'  assembly_name: GRCh38'
b'  biotype: protein_coding'
b'  db_type: core'
b'  description: B-Raf proto-oncogene, serine/threonine kinase [Source:HGNC Symbol;Acc:HGNC:1097]'
b'  display_name: BRAF'
b'  end: 140924928'
b'  id: ENSG00000157764'
b'  logic_name: ensembl_havana_gene_homo_sapiens'
b'  object_type: Gene'
b'  seq_region_name: 7'
b'  source: ensembl_havana'
b'  species: homo_sapiens'
b'  start: 140719327'
b'  strand: -1'
b'  version: 13'
b'BRCA2: '
b'  assembly_name: GRCh38'
b'  biotype: protein_coding'
b'  db_type: core'
b'  description: BRCA2 DNA repair associated [Source:HGNC Symbol;Acc:HGNC:1101]'
b'  display_name: BRCA2'
b'  end: 32400266'
b'  id: ENSG00000139618'
b'  logic_name: ensembl_havana_gene_homo_sapiens'
b'  object_type: Gene'
b'  seq_region_name: 13'
b'  source: ensembl_havana'
b'  species: homo_sapiens'
b'  start: 32315086'
b'  strand: 1'
b'  version: 15'
b'</pre></body></html>'


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]